In [ ]:
4-1

In [130]:
#两个创建variable的方法
#tf.Variable每次调用都会创建一个新的变量，
#如果变量名字相同，就在后面加
import tensorflow as tf
first_a=tf.Variable(1,name='v',
                    dtype=tf.int64)
second_a=tf.Variable(2,name='v',
                    dtype=tf.int64)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run([first_a,second_a]))

[1, 2]


In [133]:
#tf.get_variable的做法是，如果这个变量名字已经存在了
#，就拿这个变量，不再创建新的变量。
#但是需要注意的是，
#一定要在scope中，使用reuse这个选项
#不使用reuse是不能get_variable相同名字的变量的
#；而使用resue又只能在variable_scope中
with tf.variable_scope('var_scope') as scope:
    v=tf.get_variable(name='z1',shape=[1],
                     initializer=tf.zeros_initializer)
with tf.variable_scope(scope,reuse=True):
    v1=tf.get_variable(name='z1',shape=[1],
                      initializer=tf.zeros_initializer())
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
#python assert断言是声明其布尔值必须为真的判定，如果发生异常就说明表达示为假。可以理解assert断言语句为raise-if-not，
#用来测试表示式，其返回值为假，就会触发异常。
assert v==v1
print(v.name)
print(v1.name)

var_scope/z1:0
var_scope/z1:0


In [134]:
#tf.name_scope中
#对tf.get_variable不起作用，
#只对tf.Variable起作用
with tf.name_scope("my_scope"):
    v1=tf.get_variable("var1",shape=[1],
                       dtype=tf.float32)
    v2=tf.Variable(1,name='var2',
                  dtype=tf.float32)
    a=tf.add(v1,v2)
print(v1.name)
print(v2.name)
print(a.name)

var1:0
my_scope/var2:0
my_scope/Add:0


In [139]:
#tf.variable_scope中

#对tf.get_variable和tf.Variable都起作用
with tf.variable_scope('my_scope1'):
    v1=tf.get_variable('var11',shape=[1],dtype=tf.float32)
    v2=tf.Variable(1,name='var22',dtype=tf.float32)
    a=tf.add(v1,v2)
print(v1.name)
print(v2.name)
print(a.name)

my_scope1/var11:0
my_scope1_2/var22:0
my_scope1_2/Add:0


In [140]:
#这种机制允许在不用的name_scope中使用tf.get_variable来share变量，
#但是需要注意的是，一定要声明reuse：
with tf.name_scope('foo'):
    with tf.variable_scope('var_scope'):
        v=tf.get_variable('var',[1])
with tf.name_scope('bar'):
    with tf.variable_scope('var_scope',reuse=True):
        v1=tf.get_variable('var',[1])
    assert v1==v
    print(v.name)
    print(v1.name)

var_scope/var:0
var_scope/var:0


In [143]:
4-2
import pandas as pd                # 用于读取数据文件
import tensorflow as tf
import matplotlib.pyplot as plt    # 用于画图
import numpy as np                 # 用于后续计算

# 读取数据文件
df = pd.read_csv("ex2data1.txt", header=None)
train_data = df.values

# 分离特征和标签，并获取数据维数
train_X = train_data[:, :-1]
train_y = train_data[:, -1:]
feature_num = len(train_X[0])
sample_num = len(train_X)
print("Size of train_X: {}x{}".format(sample_num, feature_num))
print("Size of train_y: {}x{}".format(len(train_y), len(train_y[0])))

# 数据集
X = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

# 训练目标
W = tf.Variable(tf.zeros([feature_num, 1]))
b = tf.Variable([-.9])
#用 TensorFlow 的方式将损失函数表达出来：
#我们使用的损失函数为：
#J(θ)=−1m[∑i=1my(i)log(hθ(x(i)))+(1−y(i))log(1−hθ(x(i)))]
db = tf.matmul(X, tf.reshape(W, [-1, 1])) + b
hyp = tf.sigmoid(db)

cost0 = y * tf.log(hyp)
cost1 = (1 - y) * tf.log(1 - hyp)
cost = (cost0 + cost1) / -sample_num
loss = tf.reduce_sum(cost)

#定义使用的优化方法
#第一步是选取优化器，这里我们选择梯度下降方法；第二步是优化目标，从函数名字
#顾名思义，我们的优化目标是使得损失函数的值最小化。
#注意：此处的学习率（0.001）应当尽可能小，否则可能会出现损失计算中出现 log(0)的问题。
optimizer = tf.train.GradientDescentOptimizer(0.001)
train = optimizer.minimize(loss)

